In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
df = pd.read_csv('../data/av-10-21.csv')
av_df = pd.DataFrame(data = df)
av_df.info()

In [ ]:
av_df.head()

In [ ]:
df2 = pd.read_csv('../data/undrafted-rookie-info.csv')
undrafted_df = pd.DataFrame(data = df2)
undrafted_df.info()

In [ ]:
undrafted_df.head()

In [ ]:
df3 = pd.read_csv('../data/draft-10-21.csv', encoding = 'latin-1')
drafted_df = pd.DataFrame(data = df3)
drafted_df.info()

In [ ]:
drafted_df.head()

# Let's start to ask some general questions to get you started

1) Calculate AV by draft round over the last 10 years (AV per year?)(might need to look into wAV)
- break it down by position and maybe college/age

2) Try to find if there is a certain pick cutoff where the ensuing player AV's start to really fall off in the draft
- breakdown by position

In [ ]:
# need to join the drafted and undrafted dfs with the av df to analyze each separately

In [ ]:
# first I need to reorder columns according to the keys that I want to join on

drafted_df = drafted_df[['Player', 'ID', 'Pos', 'DrAge', 'College', 'Tm', 'Year', 'Rnd', 'Pick', 'Yr_From', 'Yr_To', 'Yrs_Strtd', 'wAV', 'G', 'GS']]
drafted_df.head()

In [ ]:
# simplify av df for joins

av_df2 = av_df.loc[:, ['Player', 'ID', 'AV']]
av_df2.head()

In [ ]:
drafted_av_df = pd.merge(drafted_df, av_df2, how='left', on='ID')
drafted_av_df

In [ ]:
#drop the name duplicate column
drafted_av_df = drafted_av_df.drop(columns = ['Player_y'])
drafted_av_df.head()

In [ ]:
drafted_av_df = drafted_av_df.rename(columns = {'Player_x': 'Name', 'wAV': 'AV_Rookie', 'AV': 'AV_Career'})
drafted_av_df.head()

In [ ]:
drafted_av_df = drafted_av_df[['Name', 'ID', 'Pos', 'DrAge', 'College', 'Tm', 'Year', 'Rnd', 'Pick', 'Yr_From', 'Yr_To', 'AV_Rookie', 'AV_Career', 'G', 'GS', 'Yrs_Strtd']]
drafted_av_df.head()

In [ ]:
undrafted_df = undrafted_df.drop(columns = ['AV.1', 'To'])
undrafted_df.head()

In [ ]:
undrafted_df = undrafted_df.rename(columns = {'From': 'Year1'})
undrafted_df.head()

In [ ]:
undrafted_av_df = pd.merge(undrafted_df, av_df, how = 'inner', on = 'ID')
undrafted_av_df.head()

In [ ]:
undrafted_av_df = undrafted_av_df.drop(columns = ['Player_y', 'From', 'Pos_y'])
undrafted_av_df = undrafted_av_df.rename(columns = {'Player_x': 'Name', 'AV_x': 'AV_Rookie', 'G_x': 'G_Rookie', 'GS_x': 'GS_Rookie', 'Pos_x': 'Pos', 'AV_y': 'AV_Career', 'To': 'Yr_End', 'G_y': 'G_Career', 'GS_y': 'GS_Career'})
undrafted_av_df.head()

In [ ]:
undrafted_av_df = undrafted_av_df[['Name', 'ID', 'Year1', 'Pos', 'Age', 'Team', 'Yr_End', 'AV_Rookie', 'AV_Career', 'G_Rookie', 'GS_Rookie', 'G_Career', 'GS_Career']]
undrafted_av_df.head()

In [ ]:
## writing both dataframes to csv to load to postgres

from pathlib import Path

#filepath = Path('out.csv')
#filepath.parent.mkdir(parents = True, exist_ok = True)
#undrafted_av_df.to_csv(filepath)

In [ ]:
#drafted_av_df.to_csv(filepath)

In [ ]:
drafted_av_df.head()

In [ ]:
##need to join salary info with undrafted df

%run salary_df.ipynb

salary_df.head()

In [ ]:
salary_df = salary_df.rename(columns = {'Player': 'Name'})
salary_df.head()

In [ ]:
undrafted_av_df = undrafted_av_df.astype({'Year1': object})
undrafted_av_df = undrafted_av_df.rename(columns = {'Year1': 'Year Signed'})
undrafted_av_df.info()

In [ ]:
undrafted_av_df = undrafted_av_df.astype({'Year Signed': int})
undrafted_av_df['Years'] = undrafted_av_df['Yr_End'] - undrafted_av_df['Year Signed']
undrafted_av_df

In [ ]:
udfa_salary_df = undrafted_av_df.merge(salary_df, how='inner', on = 'Name')
udfa_salary_df

In [ ]:
udfa_salary_df.info()

In [ ]:
udfa_salary_df = udfa_salary_df.astype({'Year Signed_x': int, 'Year Signed_y': int})

In [ ]:
udfa_salary_df = udfa_salary_df[udfa_salary_df.Value < 2600000]
udfa_salary_df = udfa_salary_df.sort_values(by=['Name', 'Year Signed_y'])

In [ ]:
udfa_salary_df = udfa_salary_df.drop_duplicates(subset = ['ID'])
udfa_salary_df.info()

In [ ]:
udfa_salary_df

In [ ]:
udfa_salary_df.head(60)

In [ ]:
udfa_salary_df = udfa_salary_df.rename(columns = {'Year Signed_x': 'Year1', 'Year Signed_y': 'Yr_Signed'})
udfa_salary_df = udfa_salary_df[['Name', 'ID', 'Year1', 'Yr_End', 'Pos', 'Age', 'AV_Rookie', 'AV_Career', 'Yr_Signed', 'Years_y', 'Value', 'APY', 'Guaranteed', 'APY as % Of Cap At Signing', 'Inflated Value', 'Inflated APY', 'Inflated Guaranteed']]
udfa_salary_df

In [ ]:
udfa_salary_df['years'] = udfa_salary_df['Yr_End'] - udfa_salary_df['Year1']
udfa_salary_df = udfa_salary_df[['Name', 'ID', 'Year1', 'Yr_End', 'years', 'Pos', 'Age', 'AV_Rookie', 'AV_Career', 'Yr_Signed', 'Years_y',  'Value', 'APY', 'Guaranteed', 'APY as % Of Cap At Signing', 'Inflated Value', 'Inflated APY', 'Inflated Guaranteed']]
udfa_salary_df

In [ ]:
udfa_salary_df['avg_av'] = udfa_salary_df['AV_Career']/udfa_salary_df['years']
udfa_salary_df = udfa_salary_df[['Name', 'ID', 'Year1', 'Yr_End', 'years', 'Pos', 'Age', 'AV_Rookie', 'AV_Career', 'avg_av', 'Yr_Signed', 'Years_y',  'Value', 'APY', 'Guaranteed', 'APY as % Of Cap At Signing', 'Inflated Value', 'Inflated APY', 'Inflated Guaranteed']]

In [ ]:
udfa_salary_df

In [ ]:
udfa_salary_df['Pos'] = udfa_salary_df['Pos'].mask((udfa_salary_df['Pos'] == 'FB') | (udfa_salary_df['Pos'] == 'RB'), 'RB')
udfa_salary_df['Pos'] = udfa_salary_df['Pos'].mask((udfa_salary_df['Pos'] == 'OL') | (udfa_salary_df['Pos'] == 'OT') | (udfa_salary_df['Pos'] == 'OG') | (udfa_salary_df['Pos'] == 'OC') | (udfa_salary_df['Pos'] == 'C') | (udfa_salary_df['Pos'] == 'T') | (udfa_salary_df['Pos'] == 'G') | (udfa_salary_df['Pos'] == 'LT') | (udfa_salary_df['Pos'] == 'LG') | (udfa_salary_df['Pos'] == 'RT') | (udfa_salary_df['Pos'] == 'RG') | (udfa_salary_df['Pos'] == 'TOG'), 'OL')
udfa_salary_df['Pos'] = udfa_salary_df['Pos'].mask((udfa_salary_df['Pos'] == 'DL') | (udfa_salary_df['Pos'] == 'OT') | (udfa_salary_df['Pos'] == 'DE') | (udfa_salary_df['Pos'] == 'DT') | (udfa_salary_df['Pos'] == 'NT') | (udfa_salary_df['Pos'] == 'RDE') | (udfa_salary_df['Pos'] == 'LDE') | (udfa_salary_df['Pos'] == 'LDT') | (udfa_salary_df['Pos'] == 'RDT'), 'DL')
udfa_salary_df['Pos'] = udfa_salary_df['Pos'].mask((udfa_salary_df['Pos'] == 'LB') | (udfa_salary_df['Pos'] == 'OLB') | (udfa_salary_df['Pos'] == 'ILB') | (udfa_salary_df['Pos'] == 'MLB') | (udfa_salary_df['Pos'] == 'LOLB') | (udfa_salary_df['Pos'] == 'ROLB') | (udfa_salary_df['Pos'] == 'LLB') | (udfa_salary_df['Pos'] == 'RLB') | (udfa_salary_df['Pos'] == 'RILB') | (udfa_salary_df['Pos'] == 'RILBLIL') | (udfa_salary_df['Pos'] == 'RLBMLB'), 'LB')
udfa_salary_df['Pos'] = udfa_salary_df['Pos'].mask((udfa_salary_df['Pos'] == 'CB') | (udfa_salary_df['Pos'] == 'RCB') | (udfa_salary_df['Pos'] == 'LCB') | (udfa_salary_df['Pos'] == 'S') | (udfa_salary_df['Pos'] == 'FS') | (udfa_salary_df['Pos'] == 'SS') | (udfa_salary_df['Pos'] == 'DB'), 'DB')
udfa_salary_df['Pos'] = udfa_salary_df['Pos'].mask((udfa_salary_df['Pos'] == 'TE') | (udfa_salary_df['Pos'] == 'TTE'), 'TE')
udfa_salary_df

In [ ]:
udfa_gntd = udfa_salary_df[udfa_salary_df.Guaranteed > 0]
udfa_gntd

In [ ]:
udfa_no_gntd = udfa_salary_df[udfa_salary_df.Guaranteed == 0]
udfa_no_gntd

In [ ]:
udfa_qb = udfa_salary_df[udfa_salary_df.Pos == 'QB']
udfa_rb = udfa_salary_df[udfa_salary_df.Pos == 'RB']
udfa_wr = udfa_salary_df[udfa_salary_df.Pos == 'WR']
udfa_te = udfa_salary_df[udfa_salary_df.Pos == 'TE']
udfa_ol = udfa_salary_df[(udfa_salary_df.Pos == 'OL') | (udfa_salary_df['Pos'] == 'OT') | (udfa_salary_df['Pos'] == 'OG') | (udfa_salary_df['Pos'] == 'OC') | (udfa_salary_df['Pos'] == 'C') | (udfa_salary_df['Pos'] == 'T') | (udfa_salary_df['Pos'] == 'G') | (udfa_salary_df['Pos'] == 'LT') | (udfa_salary_df['Pos'] == 'LG') | (udfa_salary_df['Pos'] == 'RT') | (udfa_salary_df['Pos'] == 'RG')]
udfa_dl = udfa_salary_df[(udfa_salary_df['Pos'] == 'DL') | (udfa_salary_df['Pos'] == 'OT') | (udfa_salary_df['Pos'] == 'DE') | (udfa_salary_df['Pos'] == 'DT') | (udfa_salary_df['Pos'] == 'NT') | (udfa_salary_df['Pos'] == 'RDE') | (udfa_salary_df['Pos'] == 'LDE') | (udfa_salary_df['Pos'] == 'LDT') | (udfa_salary_df['Pos'] == 'RDT')]
udfa_lb = udfa_salary_df[(udfa_salary_df['Pos'] == 'LB') | (udfa_salary_df['Pos'] == 'OLB') | (udfa_salary_df['Pos'] == 'ILB') | (udfa_salary_df['Pos'] == 'MLB') | (udfa_salary_df['Pos'] == 'LOLB') | (udfa_salary_df['Pos'] == 'ROLB') | (udfa_salary_df['Pos'] == 'LLB') | (udfa_salary_df['Pos'] == 'RLB')]
udfa_db = udfa_salary_df[(udfa_salary_df['Pos'] == 'CB') | (udfa_salary_df['Pos'] == 'RCB') | (udfa_salary_df['Pos'] == 'LCB') | (udfa_salary_df['Pos'] == 'S') | (udfa_salary_df['Pos'] == 'FS') | (udfa_salary_df['Pos'] == 'SS') | (udfa_salary_df['Pos'] == 'DB')]

In [ ]:
#filepath = Path('udfa-salary2.csv')
#filepath.parent.mkdir(parents = True, exist_ok = True)
#udfa_salary_df.to_csv(filepath)

In [ ]:
drafted_av_df

In [ ]:
drafted_av_df['Pos'] = drafted_av_df['Pos'].mask((drafted_av_df['Pos'] == 'FB') | (drafted_av_df['Pos'] == 'RB'), 'RB')
drafted_av_df['Pos'] = drafted_av_df['Pos'].mask((drafted_av_df['Pos'] == 'OL') | (drafted_av_df['Pos'] == 'OT') | (drafted_av_df['Pos'] == 'OG') | (drafted_av_df['Pos'] == 'OC') | (drafted_av_df['Pos'] == 'C') | (drafted_av_df['Pos'] == 'T') | (drafted_av_df['Pos'] == 'G') | (drafted_av_df['Pos'] == 'LT') | (drafted_av_df['Pos'] == 'LG') | (drafted_av_df['Pos'] == 'RT') | (drafted_av_df['Pos'] == 'RG'), 'OL')
drafted_av_df['Pos'] = drafted_av_df['Pos'].mask((drafted_av_df['Pos'] == 'DL') | (drafted_av_df['Pos'] == 'OT') | (drafted_av_df['Pos'] == 'DE') | (drafted_av_df['Pos'] == 'DT') | (drafted_av_df['Pos'] == 'NT') | (drafted_av_df['Pos'] == 'RDE') | (drafted_av_df['Pos'] == 'LDE') | (drafted_av_df['Pos'] == 'LDT') | (drafted_av_df['Pos'] == 'RDT'), 'DL')
drafted_av_df['Pos'] = drafted_av_df['Pos'].mask((drafted_av_df['Pos'] == 'LB') | (drafted_av_df['Pos'] == 'OLB') | (drafted_av_df['Pos'] == 'ILB') | (drafted_av_df['Pos'] == 'MLB') | (drafted_av_df['Pos'] == 'LOLB') | (drafted_av_df['Pos'] == 'ROLB') | (drafted_av_df['Pos'] == 'LLB') | (drafted_av_df['Pos'] == 'RLB'), 'LB')
drafted_av_df['Pos'] = drafted_av_df['Pos'].mask((drafted_av_df['Pos'] == 'CB') | (drafted_av_df['Pos'] == 'RCB') | (drafted_av_df['Pos'] == 'LCB') | (drafted_av_df['Pos'] == 'S') | (drafted_av_df['Pos'] == 'FS') | (drafted_av_df['Pos'] == 'SS') | (drafted_av_df['Pos'] == 'DB'), 'DB')
drafted_av_df

In [ ]:
drafted_av_df['years'] = drafted_av_df['Yr_To'] - drafted_av_df['Yr_From']
drafted_av_df

In [ ]:
drafted_av_df['avg_av'] = drafted_av_df['AV_Career']/drafted_av_df['years']
drafted_av_df

In [ ]:
drafted_av_df['avg_av'] = drafted_av_df.avg_av.fillna(0)
drafted_av_df

In [ ]:
dr_10 = drafted_av_df[drafted_av_df.Year == 2010]
dr_10

In [ ]:
dr_11 = drafted_av_df[drafted_av_df.Year == 2011]
dr_12 = drafted_av_df[drafted_av_df.Year == 2012]
dr_13 = drafted_av_df[drafted_av_df.Year == 2013]
dr_14 = drafted_av_df[drafted_av_df.Year == 2014]
dr_15 = drafted_av_df[drafted_av_df.Year == 2015]
dr_16 = drafted_av_df[drafted_av_df.Year == 2016]
dr_17 = drafted_av_df[drafted_av_df.Year == 2017]
dr_18 = drafted_av_df[drafted_av_df.Year == 2018]
dr_19 = drafted_av_df[drafted_av_df.Year == 2019]
dr_20 = drafted_av_df[drafted_av_df.Year == 2020]

In [ ]:
#ol = ['OT', 'OG', 'OC', 'C', 'T', 'G', 'LT', 'LG', 'RG', 'RT']
#dl = ['DE', 'DT', 'NT', 'RDE', 'LDE', 'LDT', 'RDT']
#lb = ['OLB', 'LB', 'ILB', 'MLB', 'LOLB', 'ROLB', 'LLB', 'RLB']
#db = CB, RCB, LCB, S, FS, SS, DB
dr_11_qb = dr_11[dr_11.Pos == 'QB']
dr_11_rb = dr_11[dr_11.Pos == 'RB']
dr_11_wr = dr_11[dr_11.Pos == 'WR']
dr_11_te = dr_11[dr_11.Pos == 'TE']
dr_11_ol = dr_11[(dr_11.Pos == 'OL') | (dr_11['Pos'] == 'OT') | (dr_11['Pos'] == 'OG') | (dr_11['Pos'] == 'OC') | (dr_11['Pos'] == 'C') | (dr_11['Pos'] == 'T') | (dr_11['Pos'] == 'G') | (dr_11['Pos'] == 'LT') | (dr_11['Pos'] == 'LG') | (dr_11['Pos'] == 'RT') | (dr_11['Pos'] == 'RG')]
dr_11_dl = dr_11[(dr_11['Pos'] == 'DL') | (dr_11['Pos'] == 'OT') | (dr_11['Pos'] == 'DE') | (dr_11['Pos'] == 'DT') | (dr_11['Pos'] == 'NT') | (dr_11['Pos'] == 'RDE') | (dr_11['Pos'] == 'LDE') | (dr_11['Pos'] == 'LDT') | (dr_11['Pos'] == 'RDT')]
dr_11_lb = dr_11[(dr_11['Pos'] == 'LB') | (dr_11['Pos'] == 'OLB') | (dr_11['Pos'] == 'ILB') | (dr_11['Pos'] == 'MLB') | (dr_11['Pos'] == 'LOLB') | (dr_11['Pos'] == 'ROLB') | (dr_11['Pos'] == 'LLB') | (dr_11['Pos'] == 'RLB')]
dr_11_db = dr_11[(dr_11['Pos'] == 'CB') | (dr_11['Pos'] == 'RCB') | (dr_11['Pos'] == 'LCB') | (dr_11['Pos'] == 'S') | (dr_11['Pos'] == 'FS') | (dr_11['Pos'] == 'SS') | (dr_11['Pos'] == 'DB')]

In [ ]:
dr_12_qb = dr_12[dr_12.Pos == 'QB']
dr_12_rb = dr_12[dr_12.Pos == 'RB']
dr_12_wr = dr_12[dr_12.Pos == 'WR']
dr_12_te = dr_12[dr_12.Pos == 'TE']
dr_12_ol = dr_12[(dr_12.Pos == 'OL') | (dr_12['Pos'] == 'OT') | (dr_12['Pos'] == 'OG') | (dr_12['Pos'] == 'OC') | (dr_12['Pos'] == 'C') | (dr_12['Pos'] == 'T') | (dr_12['Pos'] == 'G') | (dr_12['Pos'] == 'LT') | (dr_12['Pos'] == 'LG') | (dr_12['Pos'] == 'RT') | (dr_12['Pos'] == 'RG')]
dr_12_dl = dr_12[(dr_12['Pos'] == 'DL') | (dr_12['Pos'] == 'OT') | (dr_12['Pos'] == 'DE') | (dr_12['Pos'] == 'DT') | (dr_12['Pos'] == 'NT') | (dr_12['Pos'] == 'RDE') | (dr_12['Pos'] == 'LDE') | (dr_12['Pos'] == 'LDT') | (dr_12['Pos'] == 'RDT')]
dr_12_lb = dr_12[(dr_12['Pos'] == 'LB') | (dr_12['Pos'] == 'OLB') | (dr_12['Pos'] == 'ILB') | (dr_12['Pos'] == 'MLB') | (dr_12['Pos'] == 'LOLB') | (dr_12['Pos'] == 'ROLB') | (dr_12['Pos'] == 'LLB') | (dr_12['Pos'] == 'RLB')]
dr_12_db = dr_12[(dr_12['Pos'] == 'CB') | (dr_12['Pos'] == 'RCB') | (dr_12['Pos'] == 'LCB') | (dr_12['Pos'] == 'S') | (dr_12['Pos'] == 'FS') | (dr_12['Pos'] == 'SS') | (dr_12['Pos'] == 'DB')]

In [ ]:
dr_13_qb = dr_13[dr_13.Pos == 'QB']
dr_13_rb = dr_13[dr_13.Pos == 'RB']
dr_13_wr = dr_13[dr_13.Pos == 'WR']
dr_13_te = dr_13[dr_13.Pos == 'TE']
dr_13_ol = dr_13[(dr_13.Pos == 'OL') | (dr_13['Pos'] == 'OT') | (dr_13['Pos'] == 'OG') | (dr_13['Pos'] == 'OC') | (dr_13['Pos'] == 'C') | (dr_13['Pos'] == 'T') | (dr_13['Pos'] == 'G') | (dr_13['Pos'] == 'LT') | (dr_13['Pos'] == 'LG') | (dr_13['Pos'] == 'RT') | (dr_13['Pos'] == 'RG')]
dr_13_dl = dr_13[(dr_13['Pos'] == 'DL') | (dr_13['Pos'] == 'OT') | (dr_13['Pos'] == 'DE') | (dr_13['Pos'] == 'DT') | (dr_13['Pos'] == 'NT') | (dr_13['Pos'] == 'RDE') | (dr_13['Pos'] == 'LDE') | (dr_13['Pos'] == 'LDT') | (dr_13['Pos'] == 'RDT')]
dr_13_lb = dr_13[(dr_13['Pos'] == 'LB') | (dr_13['Pos'] == 'OLB') | (dr_13['Pos'] == 'ILB') | (dr_13['Pos'] == 'MLB') | (dr_13['Pos'] == 'LOLB') | (dr_13['Pos'] == 'ROLB') | (dr_13['Pos'] == 'LLB') | (dr_13['Pos'] == 'RLB')]
dr_13_db = dr_13[(dr_13['Pos'] == 'CB') | (dr_13['Pos'] == 'RCB') | (dr_13['Pos'] == 'LCB') | (dr_13['Pos'] == 'S') | (dr_13['Pos'] == 'FS') | (dr_13['Pos'] == 'SS') | (dr_13['Pos'] == 'DB')]

In [ ]:
dr_14_qb = dr_14[dr_14.Pos == 'QB']
dr_14_rb = dr_14[dr_14.Pos == 'RB']
dr_14_wr = dr_14[dr_14.Pos == 'WR']
dr_14_te = dr_14[dr_14.Pos == 'TE']
dr_14_ol = dr_14[(dr_14.Pos == 'OL') | (dr_14['Pos'] == 'OT') | (dr_14['Pos'] == 'OG') | (dr_14['Pos'] == 'OC') | (dr_14['Pos'] == 'C') | (dr_14['Pos'] == 'T') | (dr_14['Pos'] == 'G') | (dr_14['Pos'] == 'LT') | (dr_14['Pos'] == 'LG') | (dr_14['Pos'] == 'RT') | (dr_14['Pos'] == 'RG')]
dr_14_dl = dr_14[(dr_14['Pos'] == 'DL') | (dr_14['Pos'] == 'OT') | (dr_14['Pos'] == 'DE') | (dr_14['Pos'] == 'DT') | (dr_14['Pos'] == 'NT') | (dr_14['Pos'] == 'RDE') | (dr_14['Pos'] == 'LDE') | (dr_14['Pos'] == 'LDT') | (dr_14['Pos'] == 'RDT')]
dr_14_lb = dr_14[(dr_14['Pos'] == 'LB') | (dr_14['Pos'] == 'OLB') | (dr_14['Pos'] == 'ILB') | (dr_14['Pos'] == 'MLB') | (dr_14['Pos'] == 'LOLB') | (dr_14['Pos'] == 'ROLB') | (dr_14['Pos'] == 'LLB') | (dr_14['Pos'] == 'RLB')]
dr_14_db = dr_14[(dr_14['Pos'] == 'CB') | (dr_14['Pos'] == 'RCB') | (dr_14['Pos'] == 'LCB') | (dr_14['Pos'] == 'S') | (dr_14['Pos'] == 'FS') | (dr_14['Pos'] == 'SS') | (dr_14['Pos'] == 'DB')]

In [ ]:
dr_15_qb = dr_15[dr_15.Pos == 'QB']
dr_15_rb = dr_15[dr_15.Pos == 'RB']
dr_15_wr = dr_15[dr_15.Pos == 'WR']
dr_15_te = dr_15[dr_15.Pos == 'TE']
dr_15_ol = dr_15[(dr_15.Pos == 'OL') | (dr_15['Pos'] == 'OT') | (dr_15['Pos'] == 'OG') | (dr_15['Pos'] == 'OC') | (dr_15['Pos'] == 'C') | (dr_15['Pos'] == 'T') | (dr_15['Pos'] == 'G') | (dr_15['Pos'] == 'LT') | (dr_15['Pos'] == 'LG') | (dr_15['Pos'] == 'RT') | (dr_15['Pos'] == 'RG')]
dr_15_dl = dr_15[(dr_15['Pos'] == 'DL') | (dr_15['Pos'] == 'OT') | (dr_15['Pos'] == 'DE') | (dr_15['Pos'] == 'DT') | (dr_15['Pos'] == 'NT') | (dr_15['Pos'] == 'RDE') | (dr_15['Pos'] == 'LDE') | (dr_15['Pos'] == 'LDT') | (dr_15['Pos'] == 'RDT')]
dr_15_lb = dr_15[(dr_15['Pos'] == 'LB') | (dr_15['Pos'] == 'OLB') | (dr_15['Pos'] == 'ILB') | (dr_15['Pos'] == 'MLB') | (dr_15['Pos'] == 'LOLB') | (dr_15['Pos'] == 'ROLB') | (dr_15['Pos'] == 'LLB') | (dr_15['Pos'] == 'RLB')]
dr_15_db = dr_15[(dr_15['Pos'] == 'CB') | (dr_15['Pos'] == 'RCB') | (dr_15['Pos'] == 'LCB') | (dr_15['Pos'] == 'S') | (dr_15['Pos'] == 'FS') | (dr_15['Pos'] == 'SS') | (dr_15['Pos'] == 'DB')]

In [ ]:
dr_16_qb = dr_16[dr_16.Pos == 'QB']
dr_16_rb = dr_16[dr_16.Pos == 'RB']
dr_16_wr = dr_16[dr_16.Pos == 'WR']
dr_16_te = dr_16[dr_16.Pos == 'TE']
dr_16_ol = dr_16[(dr_16.Pos == 'OL') | (dr_16['Pos'] == 'OT') | (dr_16['Pos'] == 'OG') | (dr_16['Pos'] == 'OC') | (dr_16['Pos'] == 'C') | (dr_16['Pos'] == 'T') | (dr_16['Pos'] == 'G') | (dr_16['Pos'] == 'LT') | (dr_16['Pos'] == 'LG') | (dr_16['Pos'] == 'RT') | (dr_16['Pos'] == 'RG')]
dr_16_dl = dr_16[(dr_16['Pos'] == 'DL') | (dr_16['Pos'] == 'OT') | (dr_16['Pos'] == 'DE') | (dr_16['Pos'] == 'DT') | (dr_16['Pos'] == 'NT') | (dr_16['Pos'] == 'RDE') | (dr_16['Pos'] == 'LDE') | (dr_16['Pos'] == 'LDT') | (dr_16['Pos'] == 'RDT')]
dr_16_lb = dr_16[(dr_16['Pos'] == 'LB') | (dr_16['Pos'] == 'OLB') | (dr_16['Pos'] == 'ILB') | (dr_16['Pos'] == 'MLB') | (dr_16['Pos'] == 'LOLB') | (dr_16['Pos'] == 'ROLB') | (dr_16['Pos'] == 'LLB') | (dr_16['Pos'] == 'RLB')]
dr_16_db = dr_16[(dr_16['Pos'] == 'CB') | (dr_16['Pos'] == 'RCB') | (dr_16['Pos'] == 'LCB') | (dr_16['Pos'] == 'S') | (dr_16['Pos'] == 'FS') | (dr_16['Pos'] == 'SS') | (dr_16['Pos'] == 'DB')]

In [ ]:
dr_17_qb = dr_17[dr_17.Pos == 'QB']
dr_17_rb = dr_17[dr_17.Pos == 'RB']
dr_17_wr = dr_17[dr_17.Pos == 'WR']
dr_17_te = dr_17[dr_17.Pos == 'TE']
dr_17_ol = dr_17[(dr_17.Pos == 'OL') | (dr_17['Pos'] == 'OT') | (dr_17['Pos'] == 'OG') | (dr_17['Pos'] == 'OC') | (dr_17['Pos'] == 'C') | (dr_17['Pos'] == 'T') | (dr_17['Pos'] == 'G') | (dr_17['Pos'] == 'LT') | (dr_17['Pos'] == 'LG') | (dr_17['Pos'] == 'RT') | (dr_17['Pos'] == 'RG')]
dr_17_dl = dr_17[(dr_17['Pos'] == 'DL') | (dr_17['Pos'] == 'OT') | (dr_17['Pos'] == 'DE') | (dr_17['Pos'] == 'DT') | (dr_17['Pos'] == 'NT') | (dr_17['Pos'] == 'RDE') | (dr_17['Pos'] == 'LDE') | (dr_17['Pos'] == 'LDT') | (dr_17['Pos'] == 'RDT')]
dr_17_lb = dr_17[(dr_17['Pos'] == 'LB') | (dr_17['Pos'] == 'OLB') | (dr_17['Pos'] == 'ILB') | (dr_17['Pos'] == 'MLB') | (dr_17['Pos'] == 'LOLB') | (dr_17['Pos'] == 'ROLB') | (dr_17['Pos'] == 'LLB') | (dr_17['Pos'] == 'RLB')]
dr_17_db = dr_17[(dr_17['Pos'] == 'CB') | (dr_17['Pos'] == 'RCB') | (dr_17['Pos'] == 'LCB') | (dr_17['Pos'] == 'S') | (dr_17['Pos'] == 'FS') | (dr_17['Pos'] == 'SS') | (dr_17['Pos'] == 'DB')]

In [ ]:
dr_18_qb = dr_18[dr_18.Pos == 'QB']
dr_18_rb = dr_18[dr_18.Pos == 'RB']
dr_18_wr = dr_18[dr_18.Pos == 'WR']
dr_18_te = dr_18[dr_18.Pos == 'TE']
dr_18_ol = dr_18[(dr_18.Pos == 'OL') | (dr_18['Pos'] == 'OT') | (dr_18['Pos'] == 'OG') | (dr_18['Pos'] == 'OC') | (dr_18['Pos'] == 'C') | (dr_18['Pos'] == 'T') | (dr_18['Pos'] == 'G') | (dr_18['Pos'] == 'LT') | (dr_18['Pos'] == 'LG') | (dr_18['Pos'] == 'RT') | (dr_18['Pos'] == 'RG')]
dr_18_dl = dr_18[(dr_18['Pos'] == 'DL') | (dr_18['Pos'] == 'OT') | (dr_18['Pos'] == 'DE') | (dr_18['Pos'] == 'DT') | (dr_18['Pos'] == 'NT') | (dr_18['Pos'] == 'RDE') | (dr_18['Pos'] == 'LDE') | (dr_18['Pos'] == 'LDT') | (dr_18['Pos'] == 'RDT')]
dr_18_lb = dr_18[(dr_18['Pos'] == 'LB') | (dr_18['Pos'] == 'OLB') | (dr_18['Pos'] == 'ILB') | (dr_18['Pos'] == 'MLB') | (dr_18['Pos'] == 'LOLB') | (dr_18['Pos'] == 'ROLB') | (dr_18['Pos'] == 'LLB') | (dr_18['Pos'] == 'RLB')]
dr_18_db = dr_18[(dr_18['Pos'] == 'CB') | (dr_18['Pos'] == 'RCB') | (dr_18['Pos'] == 'LCB') | (dr_18['Pos'] == 'S') | (dr_18['Pos'] == 'FS') | (dr_18['Pos'] == 'SS') | (dr_18['Pos'] == 'DB')]

In [ ]:
dr_19_qb = dr_19[dr_19.Pos == 'QB']
dr_19_rb = dr_19[dr_19.Pos == 'RB']
dr_19_wr = dr_19[dr_19.Pos == 'WR']
dr_19_te = dr_19[dr_19.Pos == 'TE']
dr_19_ol = dr_19[(dr_19.Pos == 'OL') | (dr_19['Pos'] == 'OT') | (dr_19['Pos'] == 'OG') | (dr_19['Pos'] == 'OC') | (dr_19['Pos'] == 'C') | (dr_19['Pos'] == 'T') | (dr_19['Pos'] == 'G') | (dr_19['Pos'] == 'LT') | (dr_19['Pos'] == 'LG') | (dr_19['Pos'] == 'RT') | (dr_19['Pos'] == 'RG')]
dr_19_dl = dr_19[(dr_19['Pos'] == 'DL') | (dr_19['Pos'] == 'OT') | (dr_19['Pos'] == 'DE') | (dr_19['Pos'] == 'DT') | (dr_19['Pos'] == 'NT') | (dr_19['Pos'] == 'RDE') | (dr_19['Pos'] == 'LDE') | (dr_19['Pos'] == 'LDT') | (dr_19['Pos'] == 'RDT')]
dr_19_lb = dr_19[(dr_19['Pos'] == 'LB') | (dr_19['Pos'] == 'OLB') | (dr_19['Pos'] == 'ILB') | (dr_19['Pos'] == 'MLB') | (dr_19['Pos'] == 'LOLB') | (dr_19['Pos'] == 'ROLB') | (dr_19['Pos'] == 'LLB') | (dr_19['Pos'] == 'RLB')]
dr_19_db = dr_19[(dr_19['Pos'] == 'CB') | (dr_19['Pos'] == 'RCB') | (dr_19['Pos'] == 'LCB') | (dr_19['Pos'] == 'S') | (dr_19['Pos'] == 'FS') | (dr_19['Pos'] == 'SS') | (dr_19['Pos'] == 'DB')]

In [ ]:
dr_20_qb = dr_20[dr_20.Pos == 'QB']
dr_20_rb = dr_20[dr_20.Pos == 'RB']
dr_20_wr = dr_20[dr_20.Pos == 'WR']
dr_20_te = dr_20[dr_20.Pos == 'TE']
dr_20_ol = dr_20[(dr_20.Pos == 'OL') | (dr_20['Pos'] == 'OT') | (dr_20['Pos'] == 'OG') | (dr_20['Pos'] == 'OC') | (dr_20['Pos'] == 'C') | (dr_20['Pos'] == 'T') | (dr_20['Pos'] == 'G') | (dr_20['Pos'] == 'LT') | (dr_20['Pos'] == 'LG') | (dr_20['Pos'] == 'RT') | (dr_20['Pos'] == 'RG')]
dr_20_dl = dr_20[(dr_20['Pos'] == 'DL') | (dr_20['Pos'] == 'OT') | (dr_20['Pos'] == 'DE') | (dr_20['Pos'] == 'DT') | (dr_20['Pos'] == 'NT') | (dr_20['Pos'] == 'RDE') | (dr_20['Pos'] == 'LDE') | (dr_20['Pos'] == 'LDT') | (dr_20['Pos'] == 'RDT')]
dr_20_lb = dr_20[(dr_20['Pos'] == 'LB') | (dr_20['Pos'] == 'OLB') | (dr_20['Pos'] == 'ILB') | (dr_20['Pos'] == 'MLB') | (dr_20['Pos'] == 'LOLB') | (dr_20['Pos'] == 'ROLB') | (dr_20['Pos'] == 'LLB') | (dr_20['Pos'] == 'RLB')]
dr_20_db = dr_20[(dr_20['Pos'] == 'CB') | (dr_20['Pos'] == 'RCB') | (dr_20['Pos'] == 'LCB') | (dr_20['Pos'] == 'S') | (dr_20['Pos'] == 'FS') | (dr_20['Pos'] == 'SS') | (dr_20['Pos'] == 'DB')]

In [ ]:
dr_10_qb = dr_10[dr_10.Pos == 'QB']
dr_10_rb = dr_10[dr_10.Pos == 'RB']
dr_10_wr = dr_10[dr_10.Pos == 'WR']
dr_10_te = dr_10[dr_10.Pos == 'TE']
dr_10_ol = dr_10[(dr_10.Pos == 'OL') | (dr_10['Pos'] == 'OT') | (dr_10['Pos'] == 'OG') | (dr_10['Pos'] == 'OC') | (dr_10['Pos'] == 'C') | (dr_10['Pos'] == 'T') | (dr_10['Pos'] == 'G') | (dr_10['Pos'] == 'LT') | (dr_10['Pos'] == 'LG') | (dr_10['Pos'] == 'RT') | (dr_10['Pos'] == 'RG')]
dr_10_dl = dr_10[(dr_10['Pos'] == 'DL') | (dr_10['Pos'] == 'OT') | (dr_10['Pos'] == 'DE') | (dr_10['Pos'] == 'DT') | (dr_10['Pos'] == 'NT') | (dr_10['Pos'] == 'RDE') | (dr_10['Pos'] == 'LDE') | (dr_10['Pos'] == 'LDT') | (dr_10['Pos'] == 'RDT')]
dr_10_lb = dr_10[(dr_10['Pos'] == 'LB') | (dr_10['Pos'] == 'OLB') | (dr_10['Pos'] == 'ILB') | (dr_10['Pos'] == 'MLB') | (dr_10['Pos'] == 'LOLB') | (dr_10['Pos'] == 'ROLB') | (dr_10['Pos'] == 'LLB') | (dr_10['Pos'] == 'RLB')]
dr_10_db = dr_10[(dr_10['Pos'] == 'CB') | (dr_10['Pos'] == 'RCB') | (dr_10['Pos'] == 'LCB') | (dr_10['Pos'] == 'S') | (dr_10['Pos'] == 'FS') | (dr_10['Pos'] == 'SS') | (dr_10['Pos'] == 'DB')]

In [ ]:
dr_qb = drafted_av_df[drafted_av_df.Pos == 'QB']
dr_rb = drafted_av_df[drafted_av_df.Pos == 'RB']
dr_wr = drafted_av_df[drafted_av_df.Pos == 'WR']
dr_te = drafted_av_df[drafted_av_df.Pos == 'TE']
dr_ol = drafted_av_df[(drafted_av_df.Pos == 'OL') | (drafted_av_df['Pos'] == 'OT') | (drafted_av_df['Pos'] == 'OG') | (drafted_av_df['Pos'] == 'OC') | (drafted_av_df['Pos'] == 'C') | (drafted_av_df['Pos'] == 'T') | (drafted_av_df['Pos'] == 'G') | (drafted_av_df['Pos'] == 'LT') | (drafted_av_df['Pos'] == 'LG') | (drafted_av_df['Pos'] == 'RT') | (drafted_av_df['Pos'] == 'RG')]
dr_dl = drafted_av_df[(drafted_av_df['Pos'] == 'DL') | (drafted_av_df['Pos'] == 'OT') | (drafted_av_df['Pos'] == 'DE') | (drafted_av_df['Pos'] == 'DT') | (drafted_av_df['Pos'] == 'NT') | (drafted_av_df['Pos'] == 'RDE') | (drafted_av_df['Pos'] == 'LDE') | (drafted_av_df['Pos'] == 'LDT') | (drafted_av_df['Pos'] == 'RDT')]
dr_lb = drafted_av_df[(drafted_av_df['Pos'] == 'LB') | (drafted_av_df['Pos'] == 'OLB') | (drafted_av_df['Pos'] == 'ILB') | (drafted_av_df['Pos'] == 'MLB') | (drafted_av_df['Pos'] == 'LOLB') | (drafted_av_df['Pos'] == 'ROLB') | (drafted_av_df['Pos'] == 'LLB') | (drafted_av_df['Pos'] == 'RLB')]
dr_db = drafted_av_df[(drafted_av_df['Pos'] == 'CB') | (drafted_av_df['Pos'] == 'RCB') | (drafted_av_df['Pos'] == 'LCB') | (drafted_av_df['Pos'] == 'S') | (drafted_av_df['Pos'] == 'FS') | (drafted_av_df['Pos'] == 'SS') | (drafted_av_df['Pos'] == 'DB')]

In [ ]:
udfa_salary_df.info()

In [ ]:
udfa_salary_df = udfa_salary_df.dropna()
udfa_salary_df

In [ ]:
plt.bar(udfa_salary_df.Pos, udfa_salary_df.avg_av)
plt.rcParams["figure.figsize"] = (40,16)
plt.xticks(fontsize = 16, rotation = 45)
plt.yticks(fontsize = 16)
plt.show()

labels = udfa_salary_df.Pos
yay = udfa_gntd.avg_av
shoot = udfa_no_gntd.avg_av

x = np.arange(len(labels))  # the label locations
width = 0.15  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, yay, width, label='Guarantee')
rects2 = ax.bar(x + width/2, shoot, width, label='No Guarantee')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('AVG AV')
ax.set_title('AVG AV by Guaranteed Money')
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

In [ ]:
udfa_gntd

In [ ]:
udfa_gntd = udfa_gntd.dropna()
udfa_gntd

In [ ]:
udfa_no_gntd = udfa_no_gntd.dropna()
udfa_no_gntd

In [ ]:
udfa_gntd = udfa_gntd.fillna(0)
udfa_gntd

In [ ]:
udfa_no_gntd = udfa_no_gntd.fillna(0)
udfa_no_gntd

In [ ]:
plt.bar(udfa_gntd.Pos, udfa_gntd.avg_av)
plt.rcParams["figure.figsize"] = (20,8)
plt.xticks(fontsize = 16, rotation = 45)
plt.yticks(fontsize = 16)
plt.show()

In [ ]:
plt.bar(udfa_no_gntd.Pos, udfa_no_gntd.avg_av)
plt.rcParams["figure.figsize"] = (20,8)
plt.xticks(fontsize = 16, rotation = 45)
plt.yticks(fontsize = 16)
plt.show()

In [ ]:
plt.bar(drafted_av_df.Rnd, drafted_av_df.avg_av)
plt.rcParams["figure.figsize"] = (20,8)
plt.xticks(fontsize = 16, rotation = 45)
plt.yticks(fontsize = 16)
plt.show()

In [ ]:
plt.scatter(drafted_av_df.Pick, drafted_av_df.avg_av)
plt.ylabel('AVG AV')
plt.xlabel('Pick Selected')
plt.show()

In [ ]:
plt.scatter(udfa_salary_df.Guaranteed, udfa_salary_df.avg_av)
plt.ylabel('AVG AV')
plt.xlabel('Guaranteed Money')
plt.show()

In [ ]:
dr_7_rnd = drafted_av_df[drafted_av_df.Rnd == 7]
dr_7_rnd

In [ ]:
dr_1_rnd = drafted_av_df[drafted_av_df.Rnd == 1]
dr_2_rnd = drafted_av_df[drafted_av_df.Rnd == 2]
dr_3_rnd = drafted_av_df[drafted_av_df.Rnd == 3]
dr_4_rnd = drafted_av_df[drafted_av_df.Rnd == 4]
dr_5_rnd = drafted_av_df[drafted_av_df.Rnd == 5]
dr_6_rnd = drafted_av_df[drafted_av_df.Rnd == 6]

In [ ]:
dr_late_rnds = drafted_av_df[(drafted_av_df['Rnd'] == 5) | (drafted_av_df['Rnd'] ==6) | (drafted_av_df['Rnd']==7)]
dr_late_rnds

In [ ]:
filepath = Path('late-rounds.csv')
filepath.parent.mkdir(parents = True, exist_ok = True)
udfa_salary_df.to_csv(filepath)

In [ ]:
research = pd.merge(dr_late_rnds, udfa_salary_df, how = 'outer')
research.info()

In [ ]:
plt.bar(research.Rnd, research.avg_av)
plt.rcParams["figure.figsize"] = (20,8)
plt.xticks(fontsize = 16, rotation = 45)
plt.yticks(fontsize = 16)
plt.show()

In [ ]:
research['Rnd'] = research['Rnd'].fillna(0)
research

In [ ]:
plt.bar(dr_late_rnds.Pos, dr_late_rnds.avg_av)
plt.rcParams["figure.figsize"] = (20,8)
plt.xticks(fontsize = 16, rotation = 45)
plt.yticks(fontsize = 16)
plt.show()

In [ ]:
plt.bar(drafted_av_df.Rnd, drafted_av_df.avg_av)
plt.rcParams["figure.figsize"] = (20,8)
plt.xticks(fontsize = 16, rotation = 45)
plt.yticks(fontsize = 16)
plt.show()